In [ ]:
project_path = "/media/vijinim/data/Experiments/Data/FUDAN/Uniprot-12/"

# kraken2 --confidence 0.1 
# https://www.biostars.org/p/402619/

In [ ]:
import re
import xlsxwriter
import collections

from Bio import SeqIO

# Get contig lengths

In [ ]:
contig_lengths = {}

for index, record in enumerate(SeqIO.parse(project_path+"final.contigs.fa", "fasta")):
    contig_lengths[record.id] = len(record.seq)

# Get kraken result

In [ ]:
taxid_total_len = {}

species_names_taxid = {}

taxid_list = []

contig_taxid = {}

taxid_contigs = {}

taxid_to_species = {}

species_names_taxid_length = {}

with open(project_path+"MEGAHIT_new_kraken_res_0.1.txt", "r") as myfile:
    for line in myfile.readlines():
        strings = line.strip().split("\t")
        
        if strings[0] == "C":
            
            species_strings = strings[2].split(" ")
            
            if len(species_strings) > 3 and "unclassified" not in strings[2] and "complex" not in strings[2] and " group " not in strings[2] and "Human" not in strings[2] and "phage" not in strings[2] and "cellular organisms" not in strings[2]:
                
                taxid = species_strings[-1][:-1]
                
                my_species = ""
                
                if species_strings[0] == "Candidatus":
                    my_species = species_strings[0] + " " + species_strings[1] + " " + species_strings[2]
                    
                elif species_strings[1] != "sp.":
                    my_species = species_strings[0] + " " + species_strings[1]
                elif "(taxid" not in species_strings[3]:
                    my_species = strings[2].split("(taxid")[0]
                    
                my_species = my_species.replace("[", "")
                my_species = my_species.replace("]", "")
                
                if my_species != "":
                    
                    taxid_to_species[taxid] = my_species
                    contig_taxid[strings[1]] = taxid
                    
                    if taxid not in taxid_list:
                        taxid_list.append(taxid)
                        
                    if my_species in species_names_taxid:
                        species_names_taxid[my_species].add(taxid)
                    else:
                        species_names_taxid[my_species]=set([taxid])

                    if taxid not in taxid_contigs:
                        taxid_contigs[taxid] = [strings[1]]
                    else:
                        taxid_contigs[taxid].append(strings[1])
            
                    if taxid not in taxid_total_len:
                        taxid_total_len[taxid] = contig_lengths[strings[1]]
                    else:
                        taxid_total_len[taxid] += contig_lengths[strings[1]]
                        
                    if my_species not in species_names_taxid_length:
                        species_names_taxid_length[my_species] = {}
                        species_names_taxid_length[my_species][taxid] = contig_lengths[strings[1]]
                    else:
                        if taxid not in species_names_taxid_length[my_species]:
                            species_names_taxid_length[my_species][taxid] = contig_lengths[strings[1]]
                        else:
                            species_names_taxid_length[my_species][taxid] += contig_lengths[strings[1]]
                        

In [ ]:
for species in species_names_taxid_length:
    sorted_taxids = {k: v for k, v in sorted(species_names_taxid_length[species].items(), reverse=True, key=lambda item: item[1])}
    species_names_taxid_length[species] = sorted_taxids
# species_names_taxid_length

# Build MD2 database

In [ ]:
ppath = project_path
fpath = project_path+n_species+"final.contigs.fa.lst"


def get_seqs(path):
    active = ""
    
    for line in open(path):
        if line[0] == ">":
            active += line
        elif len(active) > 0 and len(line.strip()) != 0:
            active += line
        elif len(line.strip()) == 0 and len(active) > 0:
            yield active
            active = ""
    if len(active) > 0:
        yield active

In [ ]:
genes_aa_list = {}
gene_seq = {}

genes_contigs_c = {}

gene_species_mapped_c = {}

unmapped_count = 0

mapped_gene_lengths = []
unmapped_gene_lengths = []

contigs_containing_genes = []

for seq in get_seqs(fpath):
    if "_aa|" in seq:
        
        strings = seq.split("|")
        
        gene_id = strings[0][1:]
        
        has_bin = False
        
        start_n = 'k141_'
        end_n = ''
            
        contig_num = int(re.search('%s(.*)%s' % (start_n, end_n), seq.split(">")[-1].split(" ")[0]).group(1))
        
        genes_contigs_c[gene_id] = contig_num
        
        gene_seq[gene_id] = seq
        
        if 'k141_'+str(contig_num) in contig_taxid:
            
            if contig_taxid['k141_'+str(contig_num)] in taxid_to_species:
                gene_species_mapped_c[gene_id] = taxid_to_species[contig_taxid['k141_'+str(contig_num)]]
                has_bin = True
            
        if not has_bin:
            gene_species_mapped[gene_id] = "unmapped"
            unmapped_count += 1


In [ ]:
print("Total number of genes:", len(genes_contigs_c))
print("Number of genes mapped:", len(gene_species_mapped_c))
print("Number of genes unmapped:", len(genes_contigs_c) - len(gene_species_mapped_c))
print((len(gene_species_mapped_c))/len(genes_contigs_c))


In [ ]:
# Some data we want to write to the worksheet.
gene_species = {"GeneID":"Annotation"}

with open(project_path+n_species+"/final.contigs.genes.faa", "w") as aafile:
    for gene in genes_contigs:
        
        gene_id = gene
        
        if gene_id in gene_species_mapped_c:
            gene_species[gene_id] = "["+gene_species_mapped_c[gene].replace("_", " ")+"]"
        else:
            gene_species[gene_id] = "-"
            
        aafile.write(gene_seq[gene])


In [ ]:
od = collections.OrderedDict(sorted(gene_species.items()))

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook(project_path+n_species+"/"+n_species.lower()[:-1]+".contigs.genes.species.mapped.xlsx")
worksheet = workbook.add_worksheet()

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

print(len(od.items()))

# Iterate over the data and write it out row by row.
for gene, species in od.items():
    worksheet.write(row, col,     gene)
    worksheet.write(row, col + 1, species)
    row += 1
    
workbook.close()